In [4]:
# pip install protobuf==3.20.*

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

# Define the class names
class_names = ["Bird-drop", "Clean", "Dusty", "Electrical-damage", "Physical-Damage", "Snow-Covered"]

# Save class names to a file
with open('labels.txt', 'w') as f:
    for class_name in class_names:
        f.write(f"{class_name}\n")

# Define the image size and batch size
img_height, img_width = 150, 150
batch_size = 32

# Directory paths for training and validation datasets
train_dir = r'D:\solar panel dust detection\Train_Set'
val_dir = r'D:\solar panel dust detection\Test_Set'

# Image data generators for augmenting the training and validation data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 5
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)

# Save the model to an .h5 file
model.save('solar_fault_detection_model.h5')

print("Model and labels saved successfully.")


Found 885 images belonging to 6 classes.
Found 347 images belonging to 6 classes.


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 28s 809ms/step - accuracy: 0.2443 - loss: 2.0835 - val_accuracy: 0.2969 - val_loss: 1.6928
Epoch 2/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4375 - loss: 1.6208 - val_accuracy: 0.3704 - val_loss: 1.5715
Epoch 3/5


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 652ms/step - accuracy: 0.3677 - loss: 1.5483 - val_accuracy: 0.2500 - val_loss: 1.8733
Epoch 4/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3750 - loss: 1.4806 - val_accuracy: 0.3704 - val_loss: 1.3376
Epoch 5/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 640ms/step - accuracy: 0.4268 - loss: 1.4646 - val_accuracy: 0.4906 - val_loss: 1.4078


Model and labels saved successfully.


In [2]:
# import cv2
# import numpy as np
# import tensorflow as tf

# # Load the trained model
# model = tf.keras.models.load_model('solar_fault_detection_model.h5')

# # Load class names from labels.txt
# class_names = []
# with open('labels.txt', 'r') as f:
#     class_names = f.read().splitlines()

# # Initialize the camera
# cap = cv2.VideoCapture(1)

# # Check if the webcam is opened correctly
# if not cap.isOpened():
#     print("Error: Could not open camera.")
#     exit()

# # Define the image size
# img_height, img_width = 150, 150

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Convert the image to RGB and resize it
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     resized_frame = cv2.resize(rgb_frame, (img_height, img_width))

#     # Normalize the image
#     normalized_frame = resized_frame / 255.0

#     # Expand dimensions to match the input shape of the model
#     input_frame = np.expand_dims(normalized_frame, axis=0)

#     # Predict the class
#     predictions = model.predict(input_frame)
#     predicted_class = np.argmax(predictions[0])
#     class_name = class_names[predicted_class]

#     # Draw a bounding box and the class name on the original frame
#     height, width, _ = frame.shape
#     cv2.rectangle(frame, (10, 10), (width - 10, height - 10), (0, 255, 0), 2)
#     cv2.putText(frame, class_name, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

#     # Display the resulting frame
#     cv2.imshow('Solar Fault Detection', frame)

#     # Break the loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the camera and close windows
# cap.release()
# cv2.destroyAllWindows()


In [3]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pyttsx3

# Load the trained model
model = tf.keras.models.load_model('solar_fault_detection_model.h5')

# Load class names from labels.txt
class_names = []
with open('labels.txt', 'r') as f:
    class_names = f.read().splitlines()

# Initialize the camera
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Define the image size
img_height, img_width = 150, 150

previous_class_name = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB and resize it
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized_frame = cv2.resize(rgb_frame, (img_height, img_width))

    # Normalize the image
    normalized_frame = resized_frame / 255.0

    # Expand dimensions to match the input shape of the model
    input_frame = np.expand_dims(normalized_frame, axis=0)

    # Predict the class
    predictions = model.predict(input_frame)
    predicted_class = np.argmax(predictions[0])
    class_name = class_names[predicted_class]

    # If the detected class changes, speak the class name
    if class_name != previous_class_name:
        engine.say(class_name)
        engine.runAndWait()
        previous_class_name = class_name

    # Draw a bounding box and the class name on the original frame
    height, width, _ = frame.shape
    cv2.rectangle(frame, (10, 10), (width - 10, height - 10), (0, 255, 0), 2)
    cv2.putText(frame, class_name, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Solar Fault Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━